In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset

from model import GPTModel
from config import Config
from load_weights_from_gpt import load_weights_into_gpt
from gpt_download import download_and_load_gpt2
from model_generate import generate
import tiktoken

from torch.utils.data import DataLoader
import os
from pathlib import Path

import glob

/Users/abdulvajid/AI/PROJECTS/LLM-FROM-SCRATCH/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
torch.manual_seed(0);

In [3]:
config = Config({
    "n_layers": 24,
    "d_model": 1024,
    "eps": 1e-5,
    "hidden_size_multiplier": 4,
    "num_heads": 16,
    "context_len": 1024,
    "dropout": 0.01,
    "qkv_bias": True,
    "vocab_size": 50257
})

In [4]:
torch.cuda.is_available()

False

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device : {device}")

device : cpu


In [6]:
tokenizer = tiktoken.get_encoding('gpt2')

In [7]:
model = GPTModel(config=config).to(device)

In [8]:
# total parameters
total_params = 0
for params in model.parameters():
    total_params += params.numel()
    
print(total_params)

406336593


In [9]:
settigns, params = download_and_load_gpt2(model_size='355M', models_dir='gpt2')

File already exists and is up-to-date: gpt2/355M/checkpoint
File already exists and is up-to-date: gpt2/355M/encoder.json
File already exists and is up-to-date: gpt2/355M/hparams.json
File already exists and is up-to-date: gpt2/355M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2/355M/model.ckpt.index
File already exists and is up-to-date: gpt2/355M/model.ckpt.meta
File already exists and is up-to-date: gpt2/355M/vocab.bpe


In [10]:
load_weights_into_gpt(model, params)

In [11]:
print(generate(model, 
               starting_context='fibonocci code in python: def fibonacci(n):', 
               tokenizer=tokenizer, 
               max_len=40))

fibonocci code in python: def fibonacci(n): """Extract the number bounded by n from arithmetic expression IO'TIMING_RATE = b'((n % 3) % 5) % 4. This gets the scalar value of


In [12]:
data = load_dataset('teknium/openhermes')

In [13]:
data

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 242831
    })
})

In [14]:
data['train'][0]

{'output': '```perl\n#!/usr/bin/perl\n\nuse strict;\nuse warnings;\n\n# Check if a log file path is provided as command-line argument\nif (@ARGV != 1) {\n    print "Usage: $0 <log_file_path>\\n";\n    exit(1);\n}\n\nmy $log_file = $ARGV[0];\n\n# Open the log file for reading\nopen(my $fh, \'<\', $log_file) or die "Could not open file \'$log_file\' $!";\n\nmy %status_codes;\n\n# Process each line in the log file\nwhile (my $line = <$fh>) {\n    # Extract the HTTP status code from the log entry\n    if ($line =~ /HTTP\\/\\d+\\.\\d+" (\\d+)/) {\n        my $status_code = $1;\n        $status_codes{$status_code}++;\n    }\n}\n\nclose($fh);\n\n# Sort and print the results\nforeach my $code (sort { $status_codes{$b} <=> $status_codes{$a} } keys %status_codes) {\n    printf "%s: %d\\n", $code, $status_codes{$code};\n}\n```\n\nTo run this script, save it to a file named `count_http_status_codes.pl` and make sure it has executable permissions. Then, you can execute it with the following command

In [15]:
def apply_text_template(instr, input=''):
    template = """### Instruction: {instruction}
### Input: {input}
### Response:"""
    return template.format(instruction=instr, input=input)

In [16]:
def format_text_template(batch):
    template = """### Instruction: {instruction}
### Input: {input}
### Response: {output}"""
    
    return {'text': template.format(instruction=batch['instruction'], input=batch['input'], output=batch['output'])}

In [17]:
# testing map function
subset_data = data['train'].select([10,20,40])
subset_data = subset_data.map(format_text_template)

print(subset_data['text'][0])

### Instruction: Design a roller coaster with three distinct features, explaining the purpose of each feature and how it contributes to the overall ride experience.
### Input: 
### Response: 1. The Gravity-Defying Loop: One of the most iconic and thrilling features of our roller coaster is a massive, vertical loop that takes riders upside down as they travel through it at high speeds. This gravity-defying loop creates an intense sensation of weightlessness and disorientation for riders, making them feel like they are defying the laws of physics. As the train enters the loop, riders experience strong positive G-forces pushing them into their seats, followed by brief moments of weightlessness at the top of the loop before being pushed back into their seats again as they exit the loop. This feature contributes to the overall ride experience by providing a heart-pounding moment of adrenaline and excitement that leaves riders wanting more.

2. The Airtime Hills: To create a dynamic and vari

In [18]:
data = data.map(format_text_template, remove_columns=['output','input','instruction'])

In [19]:
# tokenize text in data
def tokenize(batch):
    token_ids = tokenizer.encode_batch(batch['text'], allowed_special={'<|endoftext|>'})
    return {'input_ids': token_ids}   

In [20]:
dataset = data.map(tokenize, batched=True, remove_columns=['text'])

In [21]:
# filtering any datapoint which have more that 1024 tokens
dataset = dataset.filter(lambda x: len(x['input_ids']) <= 1024)

In [22]:
tokenizer.decode(dataset['train'][0]['input_ids'])

'### Instruction: Write a Perl script that processes a log file and counts the occurrences of different HTTP status codes. The script should accept the log file path as a command-line argument and print the results to the console in descending order of frequency.\n\n### Input: \n### Response: ```perl\n#!/usr/bin/perl\n\nuse strict;\nuse warnings;\n\n# Check if a log file path is provided as command-line argument\nif (@ARGV != 1) {\n    print "Usage: $0 <log_file_path>\\n";\n    exit(1);\n}\n\nmy $log_file = $ARGV[0];\n\n# Open the log file for reading\nopen(my $fh, \'<\', $log_file) or die "Could not open file \'$log_file\' $!";\n\nmy %status_codes;\n\n# Process each line in the log file\nwhile (my $line = <$fh>) {\n    # Extract the HTTP status code from the log entry\n    if ($line =~ /HTTP\\/\\d+\\.\\d+" (\\d+)/) {\n        my $status_code = $1;\n        $status_codes{$status_code}++;\n    }\n}\n\nclose($fh);\n\n# Sort and print the results\nforeach my $code (sort { $status_codes{$b

In [23]:
dataset = dataset['train'].train_test_split(test_size=0.05, shuffle=True, seed=0)
train_dataset = dataset['train']

In [24]:
test_dataset = dataset['test'].train_test_split(test_size=0.4, seed=0)
val_dataset = test_dataset['train']
test_dataset = test_dataset['test']

In [25]:
tokenizer.eot_token

50256

In [26]:
def language_model_collate_fn(batch, max_len=None, tokenizer=tokenizer):
    # find max len seq in the batch
    pad_tok = tokenizer.eot_token
    
    batch = [seq['input_ids'] + [pad_tok] for seq in batch]
    
    batch_max_len = 0
    for seq in batch:
        if len(seq) > batch_max_len: # we give +1 since we have target token which need 1 token etc at end of token
            batch_max_len = len(seq)
            
    
    if max_len:
        max_len = max(batch_max_len, max_len)
    else:
        max_len = batch_max_len
    
    # truncate to max len
    batch = [seq[ :max_len] for seq in batch]
        
    # padding
    batch = [seq + [pad_tok] * (max_len - len(seq)) for seq in batch]
    
    # langauge modeling input_ids and output_ids
    input_ids = [seq[:-1] for seq in batch]
    target_ids = [seq[1:] for seq in batch]
    
    return torch.tensor(input_ids), torch.tensor(target_ids)

In [27]:
# # testing collate function
# language_model_collate_fn(batch=[[1,2,4,5,6],[2,3,3],[1]],
#                           tokenizer=tokenizer)

In [28]:
# initilaize dataloader
def create_dataloader(batch_size=4):
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, 
                            collate_fn=language_model_collate_fn, 
                            pin_memory=False, shuffle=True, 
                            drop_last=True)

    val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, 
                            collate_fn=language_model_collate_fn)

    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, 
                            collate_fn=language_model_collate_fn)
    return train_loader, val_loader, test_loader

In [29]:
train_dataset

Dataset({
    features: ['input_ids'],
    num_rows: 224148
})

In [30]:
test_dataset

Dataset({
    features: ['input_ids'],
    num_rows: 4720
})

In [31]:
val_dataset

Dataset({
    features: ['input_ids'],
    num_rows: 7078
})

In [32]:
train_loader, val_loader, test_loader = create_dataloader(batch_size=1)

In [33]:
# for input_ids, target_ids in train_loader:
#     break

In [34]:
# print(tokenizer.decode(input_ids[0].tolist()))

In [35]:
# model.eval();
# model.to(device);

In [36]:
# # check input to model
# with torch.no_grad():
#     output = model(input_ids[0].unsqueeze(0).to('cuda'))

In [37]:
# loss on batch
def calc_loss_on_batch(model, input_ids, target_ids):
    logits = model(input_ids).flatten(0, 1)
    target = target_ids.view(1, -1).squeeze()
    loss = torch.nn.functional.cross_entropy(logits, target)
    return loss

In [38]:
# cal loss on entire loader for eval
def cal_loss_on_data_loader(model, data_loader, max_batch=None):
    total_loss = 0
    total_batch = len(data_loader)
    
    if max_batch:
        max_batch = min(total_batch, max_batch)
    else:
        max_batch = total_batch
    
    for i, (input_ids, target_ids) in enumerate(data_loader):
        if i == max_batch:
            break
        input_ids, target_ids = input_ids.to(device), target_ids.to(device)
        loss = calc_loss_on_batch(model, input_ids, target_ids)
        total_loss += loss.item()
        
    return total_loss / max_batch
        

In [39]:
def evaluate(model, train_loader, val_loader, max_batch):
    model.eval()
    with torch.no_grad():
        train_loss = cal_loss_on_data_loader(model, train_loader, max_batch=max_batch)
        val_loss = cal_loss_on_data_loader(model, val_loader, max_batch=max_batch)
    model.train()
    return train_loss, val_loss

In [40]:
def generate(model,
            starting_context:str='what is the opposit of saying "i love you"',
            tokenizer=tokenizer,
            max_len=10,
            sampling=True,
            temperature=0.0,
            top_k=None,
            eos_id=None):
    
    
    starting_context = apply_text_template(instr=starting_context, input='')
    model.eval()
    input_ids = tokenizer.encode(starting_context)
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
    
    for i in range(max_len):
        with torch.no_grad():
            logits = model(input_ids)
            logits = logits[:,-1,:]
            
            if sampling:
                
                if top_k:
                    topk_logits, topk_pos = torch.topk(logits, k=top_k, dim=-1)
                    logits = torch.where(input=torch.tensor(float('-inf')),
                                         condition=logits < topk_logits[:,-1].reshape(-1, 1), 
                                         other=logits)
                if temperature>0.0:
                    logits = logits / temperature
                    
                probas = F.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probas, num_samples=1)
                input_ids = torch.concat([input_ids, idx_next], dim=-1)
            else:
                assert temperature==0.0 and top_k is None, "You can't set temperature or topk if sampling=False"
                last_token = torch.argmax(logits, dim=-1).unsqueeze(0)
                input_ids = torch.cat([input_ids, last_token], dim=-1)
    return tokenizer.decode(input_ids.squeeze().tolist())

In [41]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

In [42]:
# def train(model, train_loader, val_loader, optimizer, eval_freq, save_model_epoch_freq=30, save_file='./'):
#     global_step = 0
#     train_loss_history = []
#     val_loss_history = []
#     model.to(device)
    
#     for epoch, (input_ids, target_ids) in enumerate(train_loader):
#         model.train()
#         optimizer.zero_grad()
#         input_ids, target_ids = input_ids.to(device), target_ids.to(device)
#         loss = calc_loss_on_batch(model, input_ids, target_ids)
#         loss.backward()
#         optimizer.step()
        
#         global_step += 1
        
#         if global_step % eval_freq == 0 or epoch == 0:
#             train_loss, val_loss = evaluate(model, train_loader, val_loader, max_batch=10)
#             print(f"epoch: {epoch} | training_loss: {train_loss} | val_loss: {val_loss}")
#             train_loss_history.append(train_loss)
#             val_loss_history.append(val_loss)
#             print(f"Sample Generation\n{'-'*20}")
#             print(generate(model))
            
    
#     return train_loss_history,  val_loss_history
            
            

In [43]:
# train_losses, val_losses = train(model, train_loader, val_loader, optimizer, eval_freq=5, save_model_epoch_freq=10, save_file='checkpoints')

In [44]:
out = generate(model, starting_context='what is it')
print(out)

### Instruction: what is it
### Input: 
### Response: mddieet ramwool figure / backed


In [62]:
def train(model, train_loader, val_loader, optimizer, eval_freq, device, save_model_step_freq=10, save_file='runs', total_epoch=2):
    
    model.to(device)
    
    checkpoints_path = Path('runs')
    checkpoints_path.mkdir(exist_ok=True)
    model_checkpoints = list(checkpoints_path.glob("*.ckpt"))
    
    if model_checkpoints:
        last_save_path = model_checkpoints[-1].as_posix()
        print(f'Loading last saved checkpoint : {last_save_path}')
        training_state = torch.load(last_save_path)
        global_step = training_state['global_step']
        print(model.load_state_dict(training_state['model_state']))
        optimizer.load_state_dict(training_state['optimizer_state'])
        train_loss_history = training_state['train_loss_history']
        val_loss_history = training_state['val_loss_history']
        epoch = training_state['epoch']
    else:
        global_step = 0
        train_loss_history = []
        val_loss_history = []
        epoch = 0
        
    print(f'current global_step: {global_step}')
    print(f'current epoch: {epoch}')
    if train_loss_history:
        print(f'last loss: {train_loss_history[-1]} | val_loss: {val_loss_history[-1]}')
    
        
    for epoch in range(epoch, total_epoch):
        
        for input_ids, target_ids in train_loader:
            optimizer.zero_grad()
            input_ids, target_ids = input_ids.to(device), target_ids.to(device)
            loss = calc_loss_on_batch(model, input_ids, target_ids)
            loss.backward()
            optimizer.step()

            global_step += 1    
                
            if global_step % (save_model_step_freq) == 0 :
                print('saving model...')
                torch.save({'model_state' : model.state_dict(),
                            'optimizer_state': optimizer.state_dict(), 
                            'epoch': epoch,
                            'global_step':global_step,
                            'train_loss_history': train_loss_history,
                            'val_loss_history': val_loss_history},
                           (checkpoints_path / f"checkpoint.ckpt" ).as_posix())
                print('save completed..')
        
            if global_step % eval_freq == 0 or epoch == 0:
                train_loss, val_loss = evaluate(model, train_loader, val_loader, max_batch=10)
                print(f"global_step: {global_step} | training_loss: {train_loss} | val_loss: {val_loss}")
                train_loss_history.append(train_loss)
                val_loss_history.append(val_loss)
                print(f"Sample Generation\n{'-'*20}")
                print(generate(model, starting_context="what is the opposit of word 'love'"))
                
            
    
    return train_loss_history,  val_loss_history

In [63]:
train_losses, val_losses = train(model, 
                                 train_loader, 
                                 val_loader, 
                                 optimizer,
                                 total_epoch=100, 
                                 eval_freq=10, 
                                 save_model_step_freq=10, 
                                 save_file='runs', 
                                 device='cpu')

Loading last saved checkpoint : runs/checkpoint.ckpt
<All keys matched successfully>
current global_step: 510
current epoch: 5
last loss: 1.9183476209640502 | val_loss: 1.5375691652297974
saving model...
save completed..
global_step: 520 | training_loss: 1.7748191952705383 | val_loss: 1.5401386857032775
Sample Generation
--------------------
### Instruction: what is the opposit of word 'love'
### Input: 
### Response: 1. assertive
2. possessive

saving model...
save completed..
global_step: 530 | training_loss: 1.6150527238845824 | val_loss: 1.517040377855301
Sample Generation
--------------------
### Instruction: what is the opposit of word 'love'
### Input: 
### Response: love
### Input: What is the opposites
saving model...
save completed..
global_step: 540 | training_loss: 1.6841875731945037 | val_loss: 1.510129940509796
Sample Generation
--------------------
### Instruction: what is the opposit of word 'love'
### Input: 
### Response: Love is an essential development in a relation

KeyboardInterrupt: 

In [104]:
question = "who is james bond?"

In [106]:

print(generate(model, starting_context=question, max_len=50, temperature=0.7))

### Instruction: who is james bond?
### Input: 
### Response: James Bond is the title character of the James Bond film, Skyfall. Bond was the first and largest 007 to break into the world of finance. Bond joined the ranks of the super rich in his early years and became a millionaire by the time


In [103]:
print(generate(model, starting_context=question, max_len=50, sampling=False))

### Instruction: who the fuck is james bond, tell me answer in fuck
### Input: 
### Response: James Bond is a British actor, best known for playing the role of James Bond in the original James Bond film. Bond is a spy who works for the British government and is known for his loyalty to his country.<|endoftext|>The following is a list of
